# 라이블러리 불러오기

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# Retriever
- 비정형 쿼리가 주어지면 문서를 반환하는 인터페이스
- 벡터 저장소보다 더 일반적
- 문서를 저장할 필요 없이 단지 반환(또는 검색)만 가능
- 검색을 쉽게 할 수록 구성된 모듈로, 손쉽게 문서 검색

In [20]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [21]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [22]:
loader = PyPDFLoader('./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf')
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len
)
texts = text_splitter.split_documents(pages)

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {"device":"cuda:0"}
encode_kwargs = {"normalize_embeddings":True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

In [25]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

openai = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm = openai,
    chain_type="stuff",
    retriever=docsearch.as_retriever(
        search_type="mmr",
        search_kwargs={'k':3, 'fetch_k':10}
    ),
    return_source_documents=True
)
query = "혁신성장 정책금융에 대해서 설명해줘"
result = qa(query)

혁신성장 정책금융은 기업의 성장을 지원하기 위해 정부나 정책금융기관이 제공하는 금융 지원 제도입니다. 이 제도는 건강한 혁신산업 생태계를 조성하고, 기업의 성장에 필요한 자금을 지원하기 위해 시행되고 있습니다.

혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인에 따라 혁신성장 산업육성을 위한 정책금융 업무를 추진하고 있습니다. 이 가이드라인은 혁신성장 기업발굴 및 금융지원을 위해 활용하는 기준으로, '9대 테마-46개 분야-296개 품목'으로 구성되어 있습니다.

혁신성장 정책금융 제도는 공급 규모가 매년 증가하며, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원이 계속해서 추진되고 있습니다. 정책금융기관의 혁신성장 분야 정책금융 공급규모는 2017년 240,787억 원에서 연평균 37.2% 증가하여 2021년 854,338억 원에 이르는 등 그 외연을 확장하고 있습니다.

또한, 미래 성장성은 유망하지만 단기 수익 창출이 어려운 산업의 지원을 강화하기 위해 정책금융뿐만 아니라 민관주도의 역동적인 금융으로 혁신성장 금융지원 영역을 확대할 필요가 있습니다.

In [26]:
result

{'query': '혁신성장 정책금융에 대해서 설명해줘',
 'result': "혁신성장 정책금융은 기업의 성장을 지원하기 위해 정부나 정책금융기관이 제공하는 금융 지원 제도입니다. 이 제도는 건강한 혁신산업 생태계를 조성하고, 기업의 성장에 필요한 자금을 지원하기 위해 시행되고 있습니다.\n\n혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인에 따라 혁신성장 산업육성을 위한 정책금융 업무를 추진하고 있습니다. 이 가이드라인은 혁신성장 기업발굴 및 금융지원을 위해 활용하는 기준으로, '9대 테마-46개 분야-296개 품목'으로 구성되어 있습니다.\n\n혁신성장 정책금융 제도는 공급 규모가 매년 증가하며, 미래 혁신성장 분야의 글로벌 경쟁력 확보를 위한 금융지원이 계속해서 추진되고 있습니다. 정책금융기관의 혁신성장 분야 정책금융 공급규모는 2017년 240,787억 원에서 연평균 37.2% 증가하여 2021년 854,338억 원에 이르는 등 그 외연을 확장하고 있습니다.\n\n또한, 미래 성장성은 유망하지만 단기 수익 창출이 어려운 산업의 지원을 강화하기 위해 정책금융뿐만 아니라 민관주도의 역동적인 금융으로 혁신성장 금융지원 영역을 확대할 필요가 있습니다.",
 'source_documents': [Document(page_content='혁신성장 정책금융 동향 : ICT 산업을 중심으로\n  CIS이슈리포트 2022-2 호 | 3 |1. 들어가며\n▶혁신성장 정책금융기관은 건강한 혁신산업 생태계를 조성하기 위해 기업 성장에 필요한 자금을 \n지원하는 혁신성장 정책금융 제도를 시행하고 있음\n￮혁신성장 정책금융기관은 혁신성장에 대한 정의를 구체화한 정책금융 가이드라인*에 따라 혁신성장  \n산업육성을 위한 정책금융 업무를 추진 중임\n       * 혁신성장 기업발굴 및 금융지원을 위해 활용하는 기준으로 , ‘9대 테마-46개 분야-296개 품목’으로 구성\n▶혁신성장 정책금융 제도 시행 이후 공급 규모가 매년 증가하는 등, 미래 혁신성